In [1]:
# Import packages
import ee
import geemap
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Initialize google earth engine 
ee.Initialize()

In [3]:
# read in the data on fishing hours from google earth engine
fishing_hours = ee.ImageCollection('GFW/GFF/V1/fishing_hours')
# read in data on total vessel hours from google earth engine
vessel_hours = ee.ImageCollection('GFW/GFF/V1/vessel_hours')

In [4]:
# Set viz parameters
 
trawlersVis = {
  'min': 0.0,
  'max': 5.0,
  'palette': ['0d0887', '3d049b', '6903a5', '8d0fa1', 'ae2891', 'cb4679', 'df6363',
  'f0844c', 'faa638', 'fbcc27', 'f0f921']
}

In [5]:

# Define area of interest off the coast of Peru.
roi = ee.Geometry.Polygon([[-84.997101, -12.93982],
      [-84.997101, -3.79797],
      [-76.295929, -3.79797],
      [-76.295929, -12.93982],
      [-84.997101, -12.93982]], None, False)


In [6]:

# Filter by "WLD" to get all global fishing vessel hours rasters in 2016
effort = fishing_hours.filterMetadata('country', 'equals', 'WLD').filterDate('2013-01-01','2016-01-01');
purse_seines = effort.select('purse_seines')

In [7]:
# monthly sum of fishing effort
def monthly_Sum (collection, years, months):
  effort_sum = []
  for year in years:
    for month in months:
      Monthly_sum = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(month, month, 'month')) \
                              .sum() \
                              .set({'month': month, 'year': year})
      effort_sum.append (Monthly_sum)
  return ee.ImageCollection.fromImages(effort_sum)

# list of images
def monthly_images (collection, years, months):
  effort_images = []
  for year in years:
    for month in months:
      Monthly_sum = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(month, month, 'month')) \
                              .sum() \
                              .set({'month': month, 'year': year})
      effort_images.append (Monthly_sum)
  return effort_images



In [18]:
years = range(2013, 2016)
months = range(1, 12)

In [19]:
effort_collection = monthly_Sum(purse_seines, years, months)

effort_list = monthly_images(purse_seines, years, months)

effort_collection.first().getInfo()

len(effort_list)

33

In [9]:
effort_2013 = purse_seines.filter(ee.Filter.calendarRange(2013, 2013, 'year')).sum()
effort_2013 = effort_2013.reduce(ee.Reducer.sum())
effort_2013_mask = effort_2013.mask(effort_2013.gt(0)).clip(roi)
effort_2013_mask


In [10]:
effort_2014 = purse_seines.filter(ee.Filter.calendarRange(2014, 2014, 'year')).sum()
effort_2014 = effort_2014.reduce(ee.Reducer.sum())
effort_2014_mask = effort_2014.mask(effort_2014.gt(0)).clip(roi)
effort_2014_mask

In [11]:
effort_2015 = purse_seines.filter(ee.Filter.calendarRange(2015, 2015, 'year')).sum()
effort_2015 = effort_2015.reduce(ee.Reducer.sum())
effort_2015_mask = effort_2015.mask(effort_2015.gt(0)).clip(roi)
effort_2015_mask

In [12]:
effort_2016 = purse_seines.filter(ee.Filter.calendarRange(2016, 2016, 'year')).sum()
effort_2016 = effort_2016.reduce(ee.Reducer.sum())
effort_2016_mask = effort_2016.mask(effort_2016.gt(0)).clip(roi)
effort_2016_mask

In [13]:
#effort_collection = ee.ImageCollection.fromImages([effort_2013_mask, effort_2014_mask, effort_2015_mask, effort_2016_mask])

In [10]:
videoArgs = {
    'dimensions': 768,
    'region': roi,
    'framesPerSecond': 8,
    'crs': 'EPSG:3857',
    'min': 0.0,
    'max': 3.5,
    'palette': ['0d0887', '3d049b', '6903a5', '8d0fa1', 'ae2891', 'cb4679', 'df6363',
  'f0844c', 'faa638', 'fbcc27', 'f0f921']
}

In [20]:
saved_gif = os.path.join(os.path.expanduser('~'), 'eds-220/homework-2-mangrove-team/test2.gif')
geemap.download_ee_video(effort_collection, videoArgs, saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/jfrench/eds-220/homework-2-mangrove-team/test2.gif


In [21]:
geemap.show_image(saved_gif)

Output()

In [41]:
date_list = []

for i in effort_list:
    date = ee.Date(i.get('system:time_start'))
    print(date.format('GMT'))
    date_list.append(date)
    

ee.String({
  "functionInvocationValue": {
    "functionName": "Date.format",
    "arguments": {
      "date": {
        "functionInvocationValue": {
          "functionName": "Date",
          "arguments": {
            "value": {
              "functionInvocationValue": {
                "functionName": "Element.get",
                "arguments": {
                  "object": {
                    "functionInvocationValue": {
                      "functionName": "Element.set",
                      "arguments": {
                        "key": {
                          "constantValue": "year"
                        },
                        "object": {
                          "functionInvocationValue": {
                            "functionName": "Element.set",
                            "arguments": {
                              "key": {
                                "constantValue": "month"
                              },
                              "object": {
    

In [38]:
print(date_list)

len(effort_list)

[<ee.ee_string.String object at 0x7f9c5e81b3d0>, <ee.ee_string.String object at 0x7f9c5c1bb100>, <ee.ee_string.String object at 0x7f9c5c1c4370>, <ee.ee_string.String object at 0x7f9c5e7cd270>, <ee.ee_string.String object at 0x7f9c5e7cc670>, <ee.ee_string.String object at 0x7f9c5e7cf970>, <ee.ee_string.String object at 0x7f9c5e7cf4f0>, <ee.ee_string.String object at 0x7f9c5e724d00>, <ee.ee_string.String object at 0x7f9c5c0950f0>, <ee.ee_string.String object at 0x7f9c5c0d0070>, <ee.ee_string.String object at 0x7f9c5e7c4130>, <ee.ee_string.String object at 0x7f9c5e7c4d00>, <ee.ee_string.String object at 0x7f9c5e7c4e50>, <ee.ee_string.String object at 0x7f9c5e7c4700>, <ee.ee_string.String object at 0x7f9c5e7c56f0>, <ee.ee_string.String object at 0x7f9c5e7c45b0>, <ee.ee_string.String object at 0x7f9c5e7c4e20>, <ee.ee_string.String object at 0x7f9c5e7c53f0>, <ee.ee_string.String object at 0x7f9c5e7c5d50>, <ee.ee_string.String object at 0x7f9c5e7c5960>, <ee.ee_string.String object at 0x7f9c5e

33

In [24]:
out_gif = os.path.join(os.path.expanduser('~'), 'eds-220/homework-2-mangrove-team/test-label.gif')

In [25]:
geemap.add_text_to_gif(
    saved_gif,
    out_gif,
    xy=('3%', '5%'),
    text_sequence=date_list,
    font_size=30,
    font_color='#ffffff',
)

In [26]:
geemap.show_image(out_gif)

Output()